In [ ]:
import torch

In [ ]:
def bbox_to_center(bbox):
    """Convert a bounding box to the coordinates of the center of the box

    Args:
        bbox: list of the coordinates that defines the bbox ex: [xmin,ymin,xmax,ymax]
    """
    # Convert to numpy if it's a tensor
    if torch.is_tensor(bbox):
        bbox = bbox.float().cpu().numpy()  # Convert bfloat16 to float32 first

    # Find coordinates of True elements (the mask)
    x = int((bbox[0] + bbox[2]) / 2)
    y = int((bbox[1] + bbox[3]) / 2)

    return [x,y]

In [ ]:
import cv2
import os
import numpy as np

def extract(path_video, centers, padding_v, padding_h):
    """
    Extract a video centered on the object defined by the centers.

    Args:
        path_video: path to the video of interest
        centers: numpy array of coordinates of the tracked object's centers [[x1,y1],[x2,y2],...]
        padding_v: vertical padding to add
        padding_h: horizontal padding to add
    """
    cap = cv2.VideoCapture(path_video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frames = []

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # coordonnées du centre
        cx, cy = int(centers[i][0]), int(centers[i][1])

        # Ajouter un padding autour de la frame
        frame_padded = cv2.copyMakeBorder(
            frame,
            padding_v, padding_v, padding_h, padding_h,
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0]  # noir
        )

        # Ajuster le centre pour le padding
        cx_p = cx + padding_h
        cy_p = cy + padding_v

        y1 = cy_p - padding_v
        y2 = cy_p + padding_v
        x1 = cx_p - padding_h
        x2 = cx_p + padding_h

        extract_frame = frame_padded[y1:y2, x1:x2]
        frames.append(extract_frame)

    h, w = frames[0].shape[:2]
    output_path = f"{os.path.splitext(path_video)[0]}_extracted.mp4"
    out = cv2.VideoWriter(output_path,
                          cv2.VideoWriter_fourcc(*"mp4v"),
                          fps, (w, h))  # width, height

    for f in frames:
        out.write(f)
    out.release()
